In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [10]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]

y_data = [2,2,2,1,1,1,0,0]


In [11]:
y_data=np.array(pd.get_dummies(y_data))


In [12]:
# 분류기 : 3개
X=tf.placeholder('float', [None, 4])
Y=tf.placeholder('float', [None, 3])
W=tf.Variable(tf.random_normal([4,3]))
b=tf.Variable(tf.random_normal([3]))
hx=tf.nn.softmax(tf.matmul(X,W)+b)
cost=tf.reduce_mean(-tf.reduce_sum(Y+tf.log(hx), axis=1))
train=tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [13]:
from tqdm import tqdm_notebook

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in tqdm_notebook(range(2001)):
    _, cv = sess.run([train, cost], feed_dict={X:x_data, Y:y_data})
    # train을 run하면 모델 생성
    if step%200==0:
        print(step, cv)

C:\Users\poeun\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/2001 [00:00<?, ?it/s]

0 21.355442
200 16.580969
400 14.775995
600 14.477133
800 15.740231
1000 15.060213
1200 14.625165
1400 15.568745
1600 15.117209
1800 14.668413
2000 15.512439


In [16]:
res=sess.run(hx, feed_dict={X:[[1,10,7,9]]})
res # 예측값

array([[7.8752142e-05, 2.8422777e-09, 9.9992120e-01]], dtype=float32)

In [23]:
tf.argmax(res, axis=1) # 최댓값의 인덱스를 반환해준다.
sess.run(tf.argmax(res, axis=1))[0] # *=> 2 => C로 예측

2

In [61]:
xy=np.loadtxt('f:/data/data-zoo.csv', delimiter=',')
x_data = xy[:,0:-1]
y_data=xy[:,[-1]]

In [62]:
x_data.shape # (101, 16)
y_data.shape # (101, 1)

(101, 1)

In [63]:
X=tf.placeholder(tf.float32, [None, 16])
Y=tf.placeholder(tf.int32, [None, 1])

In [64]:
Y_one_hot=tf.one_hot(Y, 7) # 7가지 종류의 동물이 존재, Y값을 7자리의 수로 원핫 인코딩 해줌
# 원핫 인코딩은 integer만 가능. float은 안됨

In [65]:
Y_one_hot
#[None, 1] -> [None, 7]
#e ? : None

# 현재는 3 -> [[0,0,0,1,0,0,0]]

<tf.Tensor 'one_hot_3:0' shape=(?, 1, 7) dtype=float32>

In [66]:
Y_one_hot=tf.reshape(Y_one_hot, [-1, 7])
Y_one_hot

<tf.Tensor 'Reshape_2:0' shape=(?, 7) dtype=float32>

In [67]:
W = tf.Variable(tf.random_normal([16,7]))
b = tf.Variable(tf.random_normal([7]))

In [68]:
hx = tf.nn.softmax(tf.matmul(X,W)+b)
logits=tf.matmul(X,W)+b

In [69]:
cost =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y_one_hot))

In [70]:
train=tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [71]:
pred=tf.argmax(hx, axis=1) # 하나의 줄에 대해 가장 큰 값을 뽑아라 # pred에는 동물의 0`6번까지 들어감
acc=tf.reduce_mean(tf.cast(tf.equal(pred, tf.argmax(Y_one_hot, 1)), tf.float32))
# 두 개 인수를 같은지 다른지 보기 위해. boolena형태로 나옴 => casting(True->1, False->0)

In [77]:
y_data.flatten()
y_data.ravel()

array([0., 0., 3., 0., 0., 0., 0., 3., 3., 0., 0., 1., 3., 6., 6., 6., 1.,
       0., 3., 0., 1., 1., 0., 1., 5., 4., 4., 0., 0., 0., 5., 0., 0., 1.,
       3., 0., 0., 1., 3., 5., 5., 1., 5., 1., 0., 0., 6., 0., 0., 0., 0.,
       5., 4., 6., 0., 0., 1., 1., 1., 1., 3., 3., 2., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 6., 3., 0., 0., 2., 6., 1., 1., 2., 6., 3., 1., 0.,
       6., 3., 1., 5., 4., 2., 2., 3., 0., 0., 1., 0., 5., 0., 6., 1.])

In [79]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in tqdm_notebook(range(2001)):
        _, cv, accv = sess.run([train, cost, acc], feed_dict={X: x_data, Y: y_data})
        # if step % 100 == 0:
            # print(step, cv, accv)
    # print(sess.run(hx, feed_dict={X:x_data}))
    prediction=sess.run(pred, feed_dict={X:x_data})  # 예측값
    # print(prediction, y_data)
    for p, y in zip(prediction, y_data.flatten()):
        print('Prediction:{}, Y:{}'.format(p,int(y)))

C:\Users\poeun\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/2001 [00:00<?, ?it/s]

Prediction:0, Y:0
Prediction:0, Y:0
Prediction:3, Y:3
Prediction:0, Y:0
Prediction:0, Y:0
Prediction:0, Y:0
Prediction:0, Y:0
Prediction:3, Y:3
Prediction:3, Y:3
Prediction:0, Y:0
Prediction:0, Y:0
Prediction:1, Y:1
Prediction:3, Y:3
Prediction:6, Y:6
Prediction:6, Y:6
Prediction:6, Y:6
Prediction:1, Y:1
Prediction:0, Y:0
Prediction:3, Y:3
Prediction:0, Y:0
Prediction:1, Y:1
Prediction:1, Y:1
Prediction:0, Y:0
Prediction:1, Y:1
Prediction:5, Y:5
Prediction:4, Y:4
Prediction:4, Y:4
Prediction:0, Y:0
Prediction:0, Y:0
Prediction:0, Y:0
Prediction:5, Y:5
Prediction:0, Y:0
Prediction:0, Y:0
Prediction:1, Y:1
Prediction:3, Y:3
Prediction:0, Y:0
Prediction:0, Y:0
Prediction:1, Y:1
Prediction:3, Y:3
Prediction:5, Y:5
Prediction:5, Y:5
Prediction:1, Y:1
Prediction:5, Y:5
Prediction:1, Y:1
Prediction:0, Y:0
Prediction:0, Y:0
Prediction:6, Y:6
Prediction:0, Y:0
Prediction:0, Y:0
Prediction:0, Y:0
Prediction:0, Y:0
Prediction:5, Y:5
Prediction:4, Y:4
Prediction:6, Y:6
Prediction:0, Y:0
Prediction